In this notebook I am tuning the number of epochs and neurons in the LSTM neural net models.

In [2]:
import pandas as pd
import numpy as np
from datetime import date
bitcoin =pd.read_csv('bitcoin.csv').drop('Unnamed: 0',axis=1)
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])
bitcoin.set_index('Date',inplace=True)
bits_log_shift = (np.log(bitcoin['Price']) - np.log(bitcoin['Price']).shift()).dropna()
ethereum = pd.read_csv('ethereum.csv').drop('Unnamed: 0',axis=1)
ethereum['Date'] = pd.to_datetime(ethereum['Date'])
ethereum.set_index('Date',inplace=True)
eth_log_shift = (np.log(ethereum['Price']) - np.log(ethereum['Price']).shift()).dropna()

Step 1: Transform the time series to be stationary so it passes the dickey fuller test (extracting the signal from the noise)

Step 2 Transform the dataframes into a series of lags that can be input into the neural net model

Step 3: Put the transformed data into a scaled format [0,1] compatable with the activation functions

Step 4: Put the training set into a function that loops over each epoch that builds and returns a model

Step 5: Use the model to make a forecast

Step 6: Based upon previous min/max scaling, convert the returned values back to their original scales

Step 7: Output the predictions. Train the model on the entire time series upto the last few days (days_out), then predict the next day. Loop this again, upto until the series ends. The predictions are the predicted noise

Step 8: Plug the noise back into the signal, and calculate the root mean squared error.



In [3]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error


def Neural_Net_predictions(original_time_series, stationary_time_series, days_out,nb_epoch,neurons):
    # note all these "sub" functions are used on the stationary time series.
    # The neural nets are used to predict the noise. Once the noise is predicted
    # Its plugged back into the signal
    
    X = stationary_time_series
    
    # Step 2
    # Break the time series into shifted components. Each column is a shifted value 
    # previously in the time series
    def timeseries_to_supervised(data, lag=1):
        df = pd.DataFrame(data)
        columns = [df.shift(i) for i in range(1, lag+1)]
        columns.append(df)
        df = pd.concat(columns, axis=1)
        df.fillna(0, inplace=True)
        return df
    
    # Step 3
    # We must put the time series onto the scale acceptable by the activation functions
    def scale(train, test):
        scaler = MinMaxScaler(feature_range=(-1, 1))
        scaler = scaler.fit(train)
        train = train.reshape(train.shape[0],train.shape[1])
        train_scaled = scaler.transform(train)
        test = test.reshape(test.shape[0],test.shape[1])
        test_scaled = scaler.transform(test)
        return scaler, train_scaled, test_scaled
    
    
    # Step 4
    def fit_lstm(train, batch_size, nb_epoch, neurons):
        X, y = train[:, 0:-1], train[:, -1]
        X = X.reshape(X.shape[0], 1, X.shape[1])
        model = Sequential()
        model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        for i in range(nb_epoch):
            model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
            model.reset_states()
        return model

    
    # Step 5
    def forecast_lstm(model, batch_size, X):
        X = X.reshape(1, 1, len(X))
        yhat = model.predict(X, batch_size=batch_size)
        return yhat[0,0]
    
    #Step 6
    def invert_scale(scaler, X, value):
        new_row = [x for x in X] + [value]
        array = np.array(new_row)
        array = array.reshape(1, len(array))
        inverted = scaler.inverse_transform(array)
        return inverted[0, -1]

    # Now use all the above functions
    supervised = timeseries_to_supervised(X,1)
    supervised_values = supervised.values
    train, test = supervised_values[0:-days_out], supervised_values[-days_out:]
    scaler, train_scaled, test_scaled = scale(train, test)
    train_reshaped = train_scaled[:,0].reshape(len(train_scaled),1,1)
    lstm_model = fit_lstm(train_scaled,1,nb_epoch,neurons)
    
    #Step 7
    predictions = []
    for i in range(len(test_scaled)):
        #Make one step forecast
        X, y = test_scaled[i,0:-1], test_scaled[i,-1]
        yhat = forecast_lstm(lstm_model,1,X)
        #invert scaling
        yhat = invert_scale(scaler, X, yhat)
        #store forecast
        predictions.append(yhat)
    
    # Step 8
    # This part plugs it back into the signal
    predictions_series = pd.Series(predictions, index = original_time_series.index[-days_out:])
    a = original_time_series.loc[original_time_series.index[-(days_out+1):]]['Price']
    b = np.exp(predictions_series)
    full_predictions = pd.DataFrame(a*b,columns=['Predicted with Neural Nets']).dropna()
    df = pd.concat([original_time_series.loc[original_time_series.index[-days_out:]],full_predictions],axis=1)
    error = str(np.sqrt(mean_squared_error(df['Price'],df['Predicted with Neural Nets'])))
    #print("Neural Net Root Mean Squared Error: ",error)
    return df,error

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
#bitcoin grid
epochs_bit = [30,45]#,55,65,75]
neurons_bit = [100,125,150]#,175,200,225,250]
grid_bit = []
for e in epochs_bit:
    for n in neurons_bit:
        grid_bit.append((e,n))
        
bitcoin_errors1 = {}
for element in grid_bit:
    predictions_b, error_b =  Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=element[0],neurons=element[1])
    bitcoin_errors1[element] = [float(error_b)]
bitcoin_errors = pd.DataFrame(bitcoin_errors1).transpose()
bitcoin_errors = bitcoin_errors.reset_index().sort_values(by=0,ascending=True).rename(columns={'level_0':'epochs','level_1':'Neurons',0:'RMSE'})
bitcoin_errors = bitcoin_errors.reset_index().drop('index',axis=1)
bitcoin_errors

,epochs,Neurons,RMSE
0,45,100,45.178360
1,45,150,46.945795
2,45,125,47.017129
3,30,125,52.443253
4,30,150,54.964976
5,30,100,66.055760


In [5]:
epochs_bit = [45,55]#,65,75]
neurons_bit = [50,75,100,125,150]#,175,200,225,250]
grid_bit = []
for e in epochs_bit:
    for n in neurons_bit:
        grid_bit.append((e,n))
        
bitcoin_errors1 = {}
for element in grid_bit:
    predictions_b, error_b =  Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=element[0],neurons=element[1])
    bitcoin_errors1[element] = [float(error_b)]
bitcoin_errors = pd.DataFrame(bitcoin_errors1).transpose()
bitcoin_errors = bitcoin_errors.reset_index().sort_values(by=0,ascending=True).rename(columns={'level_0':'epochs','level_1':'Neurons',0:'RMSE'})
bitcoin_errors = bitcoin_errors.reset_index().drop('index',axis=1)
bitcoin_errors

,epochs,Neurons,RMSE
0,45,100,43.832599
1,55,100,45.647294
2,55,50,45.926413
3,45,75,46.214042
4,55,125,47.145459
5,55,75,47.884630
6,45,150,50.462438
7,45,125,53.188725
8,55,150,55.161379
9,45,50,77.559861


In [7]:
epochs_bit = [65,75]#,65,75]
neurons_bit = [50,75,100,125]#,150,175,200]#,175,200,225,250]
grid_bit = []
for e in epochs_bit:
    for n in neurons_bit:
        grid_bit.append((e,n))
        
bitcoin_errors1 = {}
for element in grid_bit:
    predictions_b, error_b =  Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=element[0],neurons=element[1])
    bitcoin_errors1[element] = [float(error_b)]
bitcoin_errors = pd.DataFrame(bitcoin_errors1).transpose()
bitcoin_errors = bitcoin_errors.reset_index().sort_values(by=0,ascending=True).rename(columns={'level_0':'epochs','level_1':'Neurons',0:'RMSE'})
bitcoin_errors = bitcoin_errors.reset_index().drop('index',axis=1)
bitcoin_errors

,epochs,Neurons,RMSE
0,65,75,43.552213
1,75,75,45.338871
2,75,50,45.881310
3,75,100,46.285710
4,75,125,46.330310
5,65,100,52.606749
6,65,50,69.070938
7,65,125,97.852062


In [ ]:
epochs_bit = [45,55]#,65,75]
neurons_bit = [50,75,100,125,150]#,175,200,225,250]
grid_bit = []
for e in epochs_bit:
    for n in neurons_bit:
        grid_bit.append((e,n))
        
bitcoin_errors1 = {}
for element in grid_bit:
    predictions_b, error_b =  Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=element[0],neurons=element[1])
    bitcoin_errors1[element] = [float(error_b)]
bitcoin_errors = pd.DataFrame(bitcoin_errors1).transpose()
bitcoin_errors = bitcoin_errors.reset_index().sort_values(by=0,ascending=True).rename(columns={'level_0':'epochs','level_1':'Neurons',0:'RMSE'})
bitcoin_errors = bitcoin_errors.reset_index().drop('index',axis=1)
bitcoin_errors

In [ ]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=30,neurons=100)
#bitcoin_NN.to_csv('bitcoin_NN_predictions_RMSE_'+bit_error+'.csv')

In [10]:
#ethereum grid
epochs_e = [55,65]
neurons_e = [150,175]
grid_eth = []
for e in epochs_e:
    for n in neurons_e:
        grid_eth.append((e,n))
        
ethereum_errors1 = {}
for element in grid_eth:
    predictions_e, error_e =  Neural_Net_predictions(ethereum,eth_log_shift,days_out=7,nb_epoch=element[0],neurons=element[1])
    ethereum_errors1[element] = [float(error_e)]
ethereum_errors = pd.DataFrame(ethereum_errors1).transpose()
ethereum_errors = ethereum_errors.reset_index().sort_values(by=0,ascending=True).rename(columns={'level_0':'epochs','level_1':'Neurons',0:'RMSE'})
ethereum_errors = ethereum_errors.reset_index().drop('index',axis=1)
ethereum_errors

,epochs,Neurons,RMSE
0,65,175,1.672472
1,65,150,1.765719
2,55,150,3.539423
3,55,175,13.086137


Note bitcoin's best lowest error currently was 30 epochs at 100 neurons with err = 43.409585

Note ethereum's best lowest error currently was 30 epochs at 150 neurons with err = 5.749

Ethereum did best at 55 epochs and 175 neurons with an error of 1.67

In [9]:
ethereum_NN, eth_error = Neural_Net_predictions(ethereum,eth_log_shift,days_out=7,nb_epoch=55,neurons=175)
ethereum_NN.to_csv('ethereum_NN_predictions_RMSE_'+eth_error+'.csv')

In [12]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=45,neurons=100)
print(bit_error)
#bitcoin_NN.to_csv('bitcoin_NN_predictions_RMSE_'+bit_error+'.csv')

46.490951727139226


In [13]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=45,neurons=200)
print(bit_error)

20.18832874321721


In [14]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=45,neurons=400)
print(bit_error)

177.5821649676008


In [15]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=45,neurons=300)
print(bit_error)

62.64744065310527


In [16]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=75,neurons=200)
print(bit_error)

46.206403678418624


In [17]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=30,neurons=200)
print(bit_error)

50.836026218358946


In [18]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=35,neurons=200)
print(bit_error)

47.54170825472671


In [19]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=45,neurons=175)
print(bit_error)

45.24576245813495


In [21]:
bitcoin_NN, bit_error = Neural_Net_predictions(bitcoin,bits_log_shift,days_out=7,nb_epoch=45,neurons=200)
bitcoin_NN.to_csv('bitcoin_NN_predictions_RMSE_'+bit_error+'.csv')